In [2]:
from Gridworld import Gridworld
game = Gridworld(size=4, mode='static')

In [3]:
game.display() # 게임화면 출력

array([['+', '-', ' ', 'P'],
       [' ', 'W', ' ', ' '],
       [' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ']], dtype='<U2')

In [4]:
game.makeMove('d')
game.makeMove('d')
game.makeMove('l')
game.display()

array([['+', '-', ' ', ' '],
       [' ', 'W', ' ', ' '],
       [' ', ' ', 'P', ' '],
       [' ', ' ', ' ', ' ']], dtype='<U2')

In [5]:
# 보상환경= [목적지(+)도착: 10, 구덩이(-)에 빠짐: -10, 그외나머지: -1]
game.reward() # 보상출력 

-1

In [6]:
game.board.render_np() #게임 상태를 출력함 (0, 플레이어), (1, 목표), (2, 구덩이), (3, 벽)

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0]],

       [[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

(4,4,4)텐서를 (64,)로 reshape하여 input데이터로 씀

In [7]:
import numpy as np
import torch
from Gridworld import Gridworld
from IPython.display import clear_output
import random
from matplotlib import pylab as plt

l1 = 64
l2 = 150
l3 = 100
l4 = 4

model = torch.nn.Sequential(
    torch.nn.Linear(64, 150),
    torch.nn.ReLU(),
    torch.nn.Linear(150, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100,4)
)
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

gamma = 0.9
epsilon = 1.0

c:\Users\dodo7\miniconda3\envs\lesa3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

In [14]:
epochs = 1000
losses = [] #손실값을 담는 리스트 history그리는데 사용됨
for i in range(epochs): #에폭수=게임횟수
    game = Gridworld(size=4, mode='static') #게임 생성
    state_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0 # 게임 state에 0~0.1사이에 난수를 더해줌(실제환경을 표방한 약간의 잡음추가)
    state = torch.from_numpy(state_).float() #Numpy배열 pytorch 텐서로 변환
    status = 1 # 게임이 진행중이면 1 아니면 0으로 수정됨
    while(status == 1): # 하나의 게임이 완료될때까지 돌아가는 루프
        qval = model(state1) # 모델 예측 시작 모든동작의 q가치 예측 (4, )
        qval_ = qval.data.numpy()
        if random.random() < epsilon: # 엡실론 확률로 무작위 액션을 취한다
            action_ = np.random.randint(0,4)
        else:
            action_ = np.argmax(qval_) # 최적의 액션 선택
        
        action = action_set[action_] # Gridworld 게임이 이해하는 동작 문자로 변환
        game.makeMove(action) # 실제로 동작 수행
        state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
        state2 = torch.from_numpy(state2_).float() #L
        reward = game.reward()
        with torch.no_grad():
            newQ = model(state2.reshape(1,64))
        maxQ = torch.max(newQ) #M
        if reward == -1: #N
            Y = reward + (gamma * maxQ)
        else:
            Y = reward
        Y = torch.Tensor([Y]).detach()
        X = qval.squeeze()[action_] #O
        loss = loss_fn(X, Y) #P
        print(i, loss.item())
        clear_output(wait=True)
        optimizer.zero_grad()
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        state1 = state2
        if reward != -1: #Q
            status = 0
    if epsilon > 0.1: #R
        epsilon -= (1/epochs)

NameError: name 'state1' is not defined

In [9]:
np.random.rand(1,64)/10.0 #D

array([[0.06431866, 0.05542547, 0.02720658, 0.03012756, 0.07448962,
        0.05998021, 0.0301201 , 0.03229061, 0.04772138, 0.02263024,
        0.04520672, 0.07324136, 0.09156831, 0.00243404, 0.05297787,
        0.0563013 , 0.0633526 , 0.02098385, 0.04694595, 0.07896645,
        0.06810451, 0.05337099, 0.07219704, 0.06064189, 0.05421194,
        0.04187904, 0.03671247, 0.09532111, 0.02916416, 0.07149972,
        0.06439372, 0.06344918, 0.05929808, 0.04041612, 0.04019605,
        0.01567657, 0.00908789, 0.0955579 , 0.08032649, 0.04987616,
        0.08431938, 0.02661148, 0.08057782, 0.04950467, 0.09650241,
        0.03893283, 0.02528349, 0.07182795, 0.03000632, 0.03164644,
        0.08178576, 0.02547021, 0.05035074, 0.00714145, 0.08161578,
        0.06702245, 0.01808934, 0.03931836, 0.00233018, 0.07458961,
        0.09295354, 0.01726657, 0.07524773, 0.06866688]])

In [12]:
game = Gridworld(size=4, mode='static') #게임 생성
# state_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0 #D

In [13]:
game.board.render_np().reshape(1,64)

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=uint8)